# Topic of Labs

This week's labs will involve some further explorations of topics we have begun in class. 

## Lab 1

 * Build a Python library our of Week 6 Lecture 2 material.  This is a Victoria Police Department crime stats library.  We start with the *vicpd.py* library, which is a collation of everything we did in Week 6, Lecture 2.  
 
 * Use **dir(vicpd)** to list the commands available in the vicpd library, or just load the file and look.
 
 * We continue exploring the VicPD and weather stats, looking to make inferences with the data. 
 

In [1]:
import vicpd as vpd

Loading the VicPD library.
[cdata] 5 years and 150 days of crime data. 86607 records total.
[ctypes] tree structure of crime types
[all_tots] totals for crime types
[all_freq] relative frequencies of crime types
[weekdaycount] loaded
[weekdaypct] loaded
[presentBDWeek] loaded
[wdatlist] 5 years and 177 days of weather data, dict of (max c, min c, mean c, rain cm, snow cm) indexed on date
VicPD library loaded.


**Suggestions**

- Make a variety of plots, of one daily crime count vs another. 

- Look for linear relationships, or perhaps non-linear relationships using least-squares. 

- Look for phenomena you might expect, like high accident counts after freezing rain. 

- If you find such, perhaps plot them spatially.  Do they occur on hills? 

In [2]:
import pprint as pp
pp.pprint(vpd.all_tots)

defaultdict(<class 'int'>,
            {'Alarm': 904,
             'Arson': 182,
             'Assault': 2215,
             'Assault with Deadly Weapon': 1208,
             'Breaking & Entering': 3616,
             'Community Policing': 34,
             'Disorder': 2673,
             'Drugs': 2720,
             'Liquor': 7894,
             'Other': 16515,
             'Property Crime': 5785,
             'Robbery': 544,
             'Theft': 14685,
             'Theft from Vehicle': 6643,
             'Theft of Vehicle': 871,
             'Traffic': 15127,
             'Vehicle Stop': 4318,
             'Weapons Offense': 673,
             ('Alarm', 'FALSE ALARMS'): 904,
             ('Arson', 'ARSON-DISREGARD FOR HUMAN LIFE'): 12,
             ('Arson', 'ARSON-PROPERTY'): 170,
             ('Assault', 'ASSAULT AGAINST POLICE OFFICER'): 300,
             ('Assault', 'ASSAULT-COMMON OR TRESPASS'): 1556,
             ('Assault', 'ASSAULTS-OTHER'): 359,
             ('Assault with Deadly 

In [ ]:
print(vpd.weekdaycount( ('Vehicle Stop', 'CYCLIST')  ))

In [ ]:
x, y = vpd.xyplot('Traffic', 'COLLISION-DAMAGE UNDER $1000', 1 )
import matplotlib.pyplot as plt
%pylab nbagg
plt.title("Min daily temp vs. number of collisions on that day")
plt.scatter(x,y)

## What we really want to do

**Question** Is look at on day $x$ if it rains, freezes that *night*, is there a disproporionate number of accidents on day $x+1$? 

**Strategy** 

 * Iterate through every day in the weather data. 
 * Look for days with rain, and min temperature < 0. 
 * Check to see if we have the crime data for the **next** day.
 * Count car accidents above and below \$1000. 
 
 * Compare these numbers to normal days. 
 
**Once you have complete this**

 * Do accidents after freezing rain happen on hills?  Intersections? Make a heat map of all the accident locations. 


## Lab 2

TBA
